In [ ]:
import os  # 운영 체제와 상호작용하기 위한 모듈
import shutil  # 파일 및 디렉토리 작업을 위한 모듈
import random  # 랜덤 작업을 위한 모듈
import scipy  # 과학 및 기술 계산을 위한 모듈
import numpy as np  # 수치 연산을 위한 Python 라이브러리
import matplotlib.pyplot as plt  # 데이터 시각화를 위한 라이브러리
from PIL import Image  # 이미지 처리 및 파일 입출력을 위한 라이브러리
from keras.preprocessing.image import ImageDataGenerator  # 이미지 데이터 생성기 유틸리티
from keras.models import Sequential, Model  # Keras의 시퀀셜 및 함수형 API 모델
from keras.layers import (Dense, Dropout, Flatten, Input, Conv2D, MaxPooling2D, 
                          BatchNormalization)  # 신경망의 레이어 구성 요소
from keras.optimizers import Adam, SGD  # 최적화 알고리즘
from keras.callbacks import EarlyStopping, ModelCheckpoint  # 학습 중 콜백 함수

def generators(train_dir, val_dir):
    """학습 및 검증 데이터를 생성하는 함수"""
    # 학습 데이터 생성기 설정
    train_datagen = ImageDataGenerator(rescale=1/255,  # 이미지의 픽셀 값을 0-1 범위로 정규화
                                       rotation_range=180,  # 최대 180도 회전
                                       width_shift_range=0.2,  # 가로 방향 이동
                                       height_shift_range=0.2,  # 세로 방향 이동
                                       shear_range=0.2,  # 시어 변환
                                       zoom_range=0.2,  # 줌 인/줌 아웃
                                       horizontal_flip=True,  # 가로 방향 뒤집기
                                       vertical_flip=True,  # 세로 방향 뒤집기
                                       fill_mode='nearest')  # 변환 중 생기는 빈 공간을 주변의 유사한 픽셀 값으로 채우기

    # 검증 데이터 생성기 설정
    val_datagen = ImageDataGenerator(rescale=1/255)  # 이미지의 픽셀 값을 0-1 범위로 정규화

    # 학습 데이터 생성
    train_generator = train_datagen.flow_from_directory(train_dir,  # 학습 데이터가 위치한 디렉토리
                                                        target_size=(227, 227),  # 입력 이미지 크기
                                                        batch_size=512,  # 배치 크기
                                                        class_mode='categorical',  # 다중 클래스 분류
                                                        shuffle=True)  # 데이터를 섞어서 배치 생성

    # 검증 데이터 생성
    val_generator = val_datagen.flow_from_directory(val_dir,  # 검증 데이터가 위치한 디렉토리
                                                    target_size=(227, 227),  # 입력 이미지 크기
                                                    batch_size=512,  # 배치 크기
                                                    class_mode='categorical')  # 다중 클래스 분류

    return train_generator, val_generator  # 학습 및 검증 데이터 생성기 반환

# 학습 및 검증 데이터 디렉토리 설정
train_dir = 'your_path'
val_dir = 'your_path'

# 데이터 생성기 생성
train_generator, val_generator = generators(train_dir, val_dir)

def alexnet():
    """AlexNet 모델을 정의하는 함수"""
    input_tensor = Input(shape=(227, 227, 3))  # 입력 텐서 정의, 입력 크기는 227x227 RGB 이미지

    # 첫 번째 Convolutional Layer
    layer = Conv2D(filters=96, kernel_size=(11, 11), strides=(4, 4), activation='relu')(input_tensor)
    layer = BatchNormalization()(layer)  # 배치 정규화
    layer = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(layer)  # 최대 풀링

    # 두 번째 Convolutional Layer
    layer = Conv2D(filters=256, kernel_size=(5, 5), strides=(1, 1), activation='relu', padding='same')(layer)
    layer = BatchNormalization()(layer)  # 배치 정규화
    layer = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(layer)  # 최대 풀링

    # 세 번째, 네 번째, 다섯 번째 Convolutional Layer
    layer = Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding='same')(layer)
    layer = BatchNormalization()(layer)  # 배치 정규화
    
    layer = Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding='same')(layer)
    layer = BatchNormalization()(layer)  # 배치 정규화

    layer = Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding='same')(layer)
    layer = BatchNormalization()(layer)  # 배치 정규화
    layer = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(layer)  # 최대 풀링
    
    layer = Flatten()(layer)  # 1차원 벡터로 평탄화

    # 첫 번째 Fully Connected Layer
    layer = Dense(units=4096, activation='relu')(layer)
    layer = Dropout(0.5)(layer)  # 드롭아웃을 적용하여 과적합 방지

    # 두 번째 Fully Connected Layer
    layer = Dense(units=4096, activation='relu')(layer)
    layer = Dropout(0.5)(layer)  # 드롭아웃을 적용하여 과적합 방지

    # 출력 Layer
    output = Dense(units=100, activation='softmax')(layer)  # 100개의 클래스에 대한 확률 분포를 출력

    # 모델 생성 및 컴파일
    model = Model(inputs=input_tensor, outputs=output)  # 입력 텐서와 출력 텐서를 사용하여 모델 정의
    model.compile(optimizer=SGD(learning_rate=0.01, momentum=0.9, decay=0.0005),  # SGD 옵티마이저 사용
                  loss='categorical_crossentropy',  # 다중 클래스 분류를 위한 손실 함수
                  metrics=['accuracy'])  # 정확도를 평가 메트릭으로 사용
    model.summary()  # 모델 구조 요약 출력
    
    return model  # 생성된 모델 반환

# AlexNet 모델 생성
model = alexnet()

# 모델 저장 경로 및 콜백 설정
model_path = 'your_path'  # 최적의 모델을 저장할 파일 경로
CP = ModelCheckpoint(filepath=model_path, monitor='val_loss', verbose=1, save_best_only=True)  # 최적의 모델을 저장하는 콜백
ES = EarlyStopping(monitor='val_loss', patience=10)  # 조기 종료를 위한 콜백, 10 에포크 동안 개선이 없으면 학습 중단

# 모델 학습
history = model.fit(train_generator,  # 학습 데이터 생성기
                    epochs=100,  # 최대 100 에포크 동안 학습
                    validation_data=val_generator,  # 검증 데이터 생성기
                    callbacks=[CP, ES])  # 콜백 함수들

# 학습 결과 시각화
plt.plot(history.history['loss'])  # 학습 손실 그래프
plt.plot(history.history['accuracy'])  # 학습 정확도 그래프
plt.plot(history.history['val_loss'])  # 검증 손실 그래프
plt.plot(history.history['val_accuracy'])  # 검증 정확도 그래프
plt.title('AlexNet')  # 그래프 제목
plt.ylabel('loss')  # y축 라벨
plt.xlabel('epoch')  # x축 라벨
plt.legend(['loss', 'val_loss', 'accuracy', 'val_accuracy'], loc='upper left')  # 범례
plt.show()  # 그래프 출력

# 최적의 검증 정확도와 손실 출력
print('Best Val Acc:', max(history.history['val_accuracy']))  # 최고 검증 정확도 출력
print('Best Val Loss:', min(history.history['val_loss']))  # 최저 검증 손실 출력
